# Вспомогательные функции

In [1]:
from typing import Callable, Tuple, List, Dict, Literal

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go
from scipy.integrate import quad
from scipy.interpolate import interp1d
from tqdm.auto import tqdm
from itertools import product

tqdm.pandas()

In [2]:
Function1 = Callable[[float], float]  # Функция одного аргумента
Function2 = Callable[[float, float], float]  # Функция двух аргументов
Segment = Tuple[float, float]  # Отрезок [a, b]
Linspace = Tuple[Segment, int]  # Отрезок [a, b] и количество точек

N = [500, 1000, 1500, 2000, 2500, 3000]  # Рассматриваемые N
ALPHA = [10 ** -i for i in range(2, 13)]

In [3]:
def get_quadrature_coefficients(linspace: Linspace, type: Literal['trapezoid', 'midpoint']) -> List[float]:
    """Находим коэффициенты квадратурной формулы трапеций"""
    segment, number_of_nodes = linspace
    _, step = np.linspace(*segment, number_of_nodes, retstep=True)

    if type == 'trapezoid':
        return [step / 2] + [step] * (number_of_nodes - 2) + [step / 2]

    return [step] * number_of_nodes

In [4]:
kernel = lambda x, s: np.cos(3 * x * s)
segment = (0, 1)

z_1 = lambda s: 1
u_1 = lambda x: np.sin(3 * x) / (3 * x) if x != 0 else 1

z_2 = lambda s: s * (1 - s)
u_2 = lambda x: (-3 * x - 3 * x * np.cos(3 * x) + 2 * np.sin(3 * x)) / (27 * x ** 3) if x != 0 else 1 / 6

In [5]:
def find_solution_using_regularization(matrix: np.ndarray, right_side: np.ndarray, alpha: float) -> np.ndarray:
    """Находит решение СЛУ с помощью регуляризации."""
    transpose_matrix = np.transpose(matrix)

    new_matrix = transpose_matrix.dot(matrix) + alpha * np.eye(matrix.shape[0])
    new_right_part = transpose_matrix.dot(right_side)

    return linalg.solve(new_matrix, new_right_part)

In [6]:
def find_integral_error(
        solution: np.ndarray,
        kernel: Function2,
        u: Function1,
        linspace: Linspace,
) -> float:
    (segment), n = linspace

    points = np.linspace(*segment, n)
    solution_poly = interp1d(points, solution)

    expected_u = lambda x: quad(lambda s, _x: kernel(_x, s) * solution_poly(s), *segment, args=(x,))[0]
    expected_u = np.vectorize(expected_u, otypes=[float])

    actual_u = np.vectorize(u, otypes=[float])
    return abs(actual_u(points) - expected_u(points)).max()

In [7]:
def plot_solutions_by_alpha(data: pd.DataFrame, segment: Segment) -> List[go.Figure]:
    figures = []

    grouped_data = data.groupby('n')
    for n in grouped_data.groups.keys():
        fig = go.Figure()

        group = grouped_data.get_group(n)
        group.apply(
            func=lambda row: fig.add_scatter(
                x=np.linspace(*segment, int(n)),
                y=eval(str(row.solution)),
                name=row.alpha,
            ),
            axis=1,
        )

        fig.update_layout(title=f'График решения для {n = }')
        figures.append(fig)

    return figures

In [8]:
def get_best_error_by_n(
        data: pd.DataFrame,
        n: int,
        drop_alphas: List[float],
        error_column: str = 'error_solve_max',
) -> pd.DataFrame:
    data = data[data.n == n]
    data = data.drop(columns=['solution'])
    data = data[~data.alpha.isin(drop_alphas)]
    return data.sort_values(error_column)

In [9]:
def plot_best_solutions(
        data: pd.DataFrame,
        n_to_best_alpha: Dict[int, float],
        segment: Segment,
        error_column: str = 'error_solve_max',
) -> go.Figure:
    fig = go.Figure()

    for n, alpha in n_to_best_alpha.items():
        row = data[(data.n == n) & (data.alpha == alpha)].iloc[0]
        print(f'error({n}, {alpha}): {row[error_column]}')
        fig.add_scatter(x=np.linspace(*segment, n), y=eval(str(row.solution)), name=f'{n = }, {alpha = }')

    fig.update_xaxes(title='x')
    fig.update_yaxes(title='y', zeroline=False)
    fig.update_layout(title='Наилучшие решения для каждого n и alpha', template=None)
    
    return fig

# Метод 1

In [10]:
def benchmark_first_method(
        kernel: Function2,
        u: Function1,
        linspace: Linspace,
        alphas: List[float],
        true_function: Function1,
) -> pd.Series:
    segment, n = linspace
    u = np.vectorize(u, otypes=[float])
    true_function = np.vectorize(true_function, otypes=[float])

    points, step = np.linspace(*segment, n, retstep=True)

    quadrature_coefficients = get_quadrature_coefficients(linspace, type='midpoint')
    matrix = [
        [coefficient * kernel(s_j, s_k) for coefficient, s_k in zip(quadrature_coefficients, points)]
        for s_j in points
    ]
    right_side = u(points)

    matrix = np.array(matrix)
    right_side = np.array(right_side)

    errors_true_max = []
    errors_solve_max = []
    solutions = []
    for alpha in tqdm(alphas, desc='alphas', leave=False):
        solution = find_solution_using_regularization(matrix, right_side, alpha)

        error_true_max = abs(solution - true_function(points)).max()
        error_solve_max = abs(matrix.dot(solution) - right_side).max()

        errors_true_max.append(error_true_max)
        errors_solve_max.append(error_solve_max)
        solutions.append(solution.tolist())

    return pd.Series([alphas, errors_true_max, errors_solve_max, solutions])

# $$z = 1$$

In [11]:
# first_method_data = pd.DataFrame(N, columns=['n'])
#
# columns_to_add = [
#     'alpha',
#     'error_true_max',
#     'error_solve_max',
#     'solution',
# ]
#
# first_method_data[columns_to_add] = first_method_data.progress_apply(
#     lambda row: benchmark_first_method(kernel, u_1, (segment, int(row.n)), ALPHAS, z_1),
#     axis=1,
# )
#
# first_method_data = first_method_data.explode(columns_to_add)
# first_method_data.to_csv('./first_z_1.csv', index=False)

In [12]:
first_method_data = pd.read_csv('first_z_1.csv')
first_method_data

,n,alpha,error_true_max,error_solve_max,solution
0,500,1.000000e-02,0.119790,2.560809e-02,"[0.978137941979886, 0.9781384928666949, 0.9781..."
1,500,1.000000e-03,0.057437,2.869070e-03,"[0.9824162121856819, 0.9824169494841672, 0.982..."
2,500,1.000000e-04,0.026765,3.334087e-04,"[0.9912740157712296, 0.991274349513722, 0.9912..."
3,500,1.000000e-05,0.017361,3.422349e-05,"[0.9944800340980404, 0.9944802109970067, 0.994..."
4,500,1.000000e-06,0.016982,3.482740e-06,"[0.9951290818044091, 0.9951292159611435, 0.995..."
...,...,...,...,...,...
61,3000,1.000000e-08,0.003949,3.344893e-08,"[0.999520289669191, 0.999520250636745, 0.99952..."
62,3000,1.000000e-09,0.003995,1.141330e-08,"[0.9995090437443008, 0.9995086558465074, 0.999..."
63,3000,1.000000e-10,0.004391,1.150994e-08,"[0.9994015268730138, 0.9993976569200805, 0.999..."
64,3000,1.000000e-11,0.006035,5.423808e-09,"[0.9989515534943187, 0.9989102199383036, 0.998..."


In [13]:
for fig in plot_solutions_by_alpha(first_method_data, segment):
    fig.show()

Unsupported

In [14]:
get_best_error_by_n(first_method_data, N[5], ALPHA[(6 - 1):], error_column='error_true_max')

,n,alpha,error_true_max,error_solve_max
59,3000,0.000001,0.004083,0.000004
58,3000,0.000010,0.005517,0.000037
57,3000,0.000100,0.016992,0.000350
56,3000,0.001000,0.052937,0.002930
55,3000,0.010000,0.117999,0.025776


In [15]:
fig = plot_best_solutions(
    first_method_data,
    {
        N[0]: 1e-6,
        N[1]: 1e-6,
        N[2]: 1e-6,
        N[3]: 1e-6,
        N[4]: 1e-6,
        N[5]: 1e-6,
    },
    segment,
    error_column='error_true_max',
)
fig.show()
fig.write_image('first_z_1.svg', height=350)

error(500, 1e-06): 0.016981775794891
error(1000, 1e-06): 0.0092682359159744
error(1500, 1e-06): 0.0066798555881608
error(2000, 1e-06): 0.0053824262516508
error(2500, 1e-06): 0.0046029304897895
error(3000, 1e-06): 0.0040828348779801


Unsupported

In [16]:
# fig = plot_best_solutions(
#     first_method_data,
#     {
#         N[5]: 1e-6,
#         N[1]: 1e-12,
#     },
#     segment,
#     error_column='error_true_max',
# )
# fig.update_layout(title='Пример "нормального" и "расшатавшегося" решения', showlegend=False)
# fig.show()
# fig.write_image('solutions_example.svg', height=400)

# $$z = s(1 - s)$$

In [17]:
# first_method_data = pd.DataFrame(N, columns=['n'])
#
# columns_to_add = [
#     'alpha',
#     'error_true_max',
#     'error_solve_max',
#     'solution',
# ]
#
# first_method_data[columns_to_add] = first_method_data.progress_apply(
#     lambda row: benchmark_first_method(kernel, u_2, (segment, int(row.n)), ALPHAS, z_2),
#     axis=1,
# )
#
# first_method_data = first_method_data.explode(columns_to_add)
# first_method_data.to_csv('./first_z_2.csv', index=False)

In [18]:
first_method_data = pd.read_csv('first_z_2.csv')
first_method_data

,n,alpha,error_true_max,error_solve_max,solution
0,500,1.000000e-02,0.171176,5.185924e-03,"[0.1711759524775745, 0.17117601819089173, 0.17..."
1,500,1.000000e-03,0.155632,2.886917e-03,"[0.1556316011196577, 0.1556325224434287, 0.155..."
2,500,1.000000e-04,0.120625,8.203757e-04,"[0.12062482573972352, 0.12062749477766098, 0.1..."
3,500,1.000000e-05,0.107886,1.270577e-04,"[0.10788588625186524, 0.10788920249341087, 0.1..."
4,500,1.000000e-06,0.103569,3.674045e-05,"[0.10356935976796637, 0.10357301156374833, 0.1..."
...,...,...,...,...,...
61,3000,1.000000e-08,0.083539,2.371021e-06,"[0.08353935136000766, 0.08353950654645505, 0.0..."
62,3000,1.000000e-09,0.081837,4.544623e-07,"[0.08183671593814872, 0.0818368918467414, 0.08..."
63,3000,1.000000e-10,0.079852,2.245380e-07,"[0.0798520822535665, 0.07985242790446556, 0.07..."
64,3000,1.000000e-11,0.072246,9.750195e-08,"[0.07224636692646846, 0.07224626889948038, 0.0..."


In [19]:
for fig in plot_solutions_by_alpha(first_method_data, segment):
    fig.show()

Unsupported

In [20]:
get_best_error_by_n(first_method_data, N[5], ALPHA[(10 - 1):], error_column='error_true_max')

,n,alpha,error_true_max,error_solve_max
63,3000,1.000000e-10,0.079852,2.245380e-07
62,3000,1.000000e-09,0.081837,4.544623e-07
61,3000,1.000000e-08,0.083539,2.371021e-06
60,3000,1.000000e-07,0.092319,1.395349e-05
59,3000,1.000000e-06,0.103915,3.693853e-05
58,3000,1.000000e-05,0.108201,1.281535e-04
57,3000,1.000000e-04,0.121085,8.267761e-04
56,3000,1.000000e-03,0.156110,2.886973e-03
55,3000,1.000000e-02,0.171474,5.182733e-03


In [21]:
fig = plot_best_solutions(
    first_method_data,
    {
        N[0]: 1e-10,
        N[1]: 1e-10,
        N[2]: 1e-10,
        N[3]: 1e-9,
        N[4]: 1e-9,
        N[5]: 1e-9,
    },
    segment,
    error_column='error_true_max',
)
fig.show()
fig.write_image('first_z_2.svg', height=350)

error(500, 1e-10): 0.079513839645598
error(1000, 1e-10): 0.0797165303870054
error(1500, 1e-10): 0.0797835065645968
error(2000, 1e-09): 0.081807290670358
error(2500, 1e-09): 0.0818246453241264
error(3000, 1e-09): 0.0818367159381487


Unsupported

# Метод №2

In [22]:
def banchmark_second_method(
        kernel: Function2,
        u: Function1,
        p: Function1,
        r: Function1,
        z_0: float,
        z_n: float,
        linspace: Linspace,
        alphas: List[float],
        true_function: Function1,
):
    segment, n = linspace
    kernel_1 = lambda s, t: quad(lambda x: kernel(x, s) * kernel(x, t), *segment)[0]
    f = lambda s: quad(lambda t: kernel(t, s) * u(t), *segment)[0]

    points, step = np.linspace(*segment, n, retstep=True)
    half_step = step / 2

    quadrature_coefficients = get_quadrature_coefficients(linspace, type='trapezoid')

    D = []
    for i in tqdm(range(1, n - 1), desc='D', leave=False):
        a = -1 * p(points[i] - half_step) / step ** 2
        b = (p(points[i] - half_step) + p(points[i] + half_step)) / step ** 2 + r(points[i])
        c = -1 * p(points[i] + half_step) / step ** 2

        if i == 1:
            D.append([b, c] + [0] * (n - 4))
        elif i == (n - 2):
            D.append([0] * (n - 4) + [a, b])
        else:
            D.append([0] * (i - 2) + [a, b, c] + [0] * (n - i - 3))

    L = [
        [quadrature_coefficients[j] * kernel_1(points[i], points[j]) for j in range(1, n - 1)]
        for i in tqdm(range(1, n - 1), desc='L', leave=False)
    ]

    F = [
        f(points[i])
        - quadrature_coefficients[0] * kernel_1(points[i], points[0]) * z_0
        - quadrature_coefficients[n - 1] * kernel_1(points[i], points[n - 1]) * z_n
        for i in tqdm(range(1, n - 1), desc='F', leave=False)
    ]

    errors_true_max = []
    errors_solve_max = []
    solutions = []
    for alpha in tqdm(alphas, desc='alphas', leave=False):
        F[0] += p(points[1] - half_step) / step ** 2 * z_0 * alpha
        F[-1] += p(points[n - 2] + half_step) / step ** 2 * z_n * alpha

        matrix = alpha * np.array(D) + np.array(L)
        right_side = np.array(F)

        solution = linalg.solve(matrix, right_side)

        error_solve_max = abs(matrix.dot(solution) - right_side).max()

        solution = np.array([z_0] + solution.tolist() + [z_n])
        error_true_max = abs(solution - true_function(points)).max()

        errors_true_max.append(error_true_max)
        errors_solve_max.append(error_solve_max)
        solutions.append(solution.tolist())

    return pd.Series([alphas, errors_true_max, errors_solve_max, solutions])

In [23]:
# second_method_data = pd.DataFrame([100], columns=['n'])

# columns_to_add = [
#     'alpha',
#     'error_true_max',
#     'error_solve_max',
#     'solution',
# ]

# second_method_data[columns_to_add] = second_method_data.progress_apply(
#     func=lambda row: banchmark_second_method(
#         kernel,
#         u_1,
#         lambda x: 1,
#         lambda x: 1,
#         1,
#         1,
#         (segment, int(row.n)),
#         [10 ** -i for i in range(2, 13)],
#         z_1,
#     ),
#     axis=1,
# )

# second_method_data = second_method_data.explode(columns_to_add)
# second_method_data.to_csv('./second_z_1.csv', index=False)

# $$z=s(1-s)$$

In [24]:
second_method_data = pd.read_csv('second_z_2.csv')
second_method_data

,n,alpha,error_true_max,error_solve_max,solution
0,500,1.000000e-02,0.057500,1.433867e-12,"[0.0, 0.0017619363895029845, 0.003515327794822..."
1,500,1.000000e-03,0.010779,1.274675e-13,"[0.0, 0.0020503087939499572, 0.004091347424412..."
2,500,1.000000e-04,0.001379,2.181588e-14,"[0.0, 0.00202815125003691, 0.00404755389055348..."
3,500,1.000000e-05,0.000468,2.414735e-15,"[0.0, 0.0020210821726985824, 0.004033525552097..."
4,500,1.000000e-06,0.000323,2.914335e-16,"[0.0, 0.0020166850447855787, 0.004024798333663..."
...,...,...,...,...,...
61,3000,1.000000e-08,0.000258,1.137979e-15,"[0.0, 0.0003357353342096189, 0.000671234250443..."
62,3000,1.000000e-09,0.000210,2.914335e-16,"[0.0, 0.00033544538246363647, 0.00067065530826..."
63,3000,1.000000e-10,0.000143,2.775558e-16,"[0.0, 0.00033496699884583473, 0.00066970013255..."
64,3000,1.000000e-11,0.000127,1.110223e-16,"[0.0, 0.00033482986382072245, 0.00066942632019..."


In [25]:
for fig in plot_solutions_by_alpha(second_method_data, segment):
    fig.show()

Unsupported

In [26]:
get_best_error_by_n(second_method_data, N[0], [], error_column='error_true_max')

,n,alpha,error_true_max,error_solve_max
10,500,1.000000e-12,0.000122,6.938894e-17
9,500,1.000000e-11,0.000125,8.326673e-17
8,500,1.000000e-10,0.000143,5.551115e-17
7,500,1.000000e-09,0.000211,5.551115e-17
6,500,1.000000e-08,0.000258,6.938894e-17
5,500,1.000000e-07,0.000270,6.938894e-17
4,500,1.000000e-06,0.000323,2.914335e-16
3,500,1.000000e-05,0.000468,2.414735e-15
2,500,1.000000e-04,0.001379,2.181588e-14
1,500,1.000000e-03,0.010779,1.274675e-13


In [27]:
fig = plot_best_solutions(
    second_method_data,
    {
        N[0]: 1e-12,
        N[1]: 1e-12,
        N[2]: 1e-12,
        N[3]: 1e-12,
        N[4]: 1e-12,
        N[5]: 1e-12,
    },
    segment,
    error_column='error_true_max',
)
fig.show()
fig.write_image('second_z_2.svg', height=350)

error(500, 1e-12): 0.0001217534058415
error(1000, 1e-12): 0.0001234925931501
error(1500, 1e-12): 0.0001238082145091
error(2000, 1e-12): 0.000123923901114
error(2500, 1e-12): 0.000123974635152
error(3000, 1e-12): 0.0001240023491469


Unsupported

In [28]:
# second_method_improved_data = pd.DataFrame(product([0, 1, 10, 100], [0, 1, 10, 100]), columns=['p', 'r'])

# columns_to_add = [
#     'alpha',
#     'error_true_max',
#     'error_solve_max',
#     'solution',
# ]

# second_method_improved_data[columns_to_add] = second_method_improved_data.progress_apply(
#     func=lambda row: banchmark_second_method(
#         kernel,
#         u_2,
#         lambda x: row.p,
#         lambda x: row.r,
#         0,
#         0,
#         (segment, 500),
#         [10 ** -12],
#         z_2,
#     ),
#     axis=1,
# )

# second_method_improved_data = second_method_improved_data.explode(columns_to_add)
# second_method_improved_data.to_csv('./second_z_2_improved.csv', index=False)

In [29]:
second_method_improved_data = pd.read_csv('second_z_2_improved.csv')
second_method_improved_data

,p,r,alpha,error_true_max,error_solve_max,solution
0,0,0,1.000000e-12,138.342190,9.575674e-16,"[0.0, -80.03525608551277, -74.21323928250922, ..."
1,0,1,1.000000e-12,0.065016,4.163336e-17,"[0.0, 0.06701568561175777, 0.0670594509007286,..."
2,0,10,1.000000e-12,0.070741,6.938894e-17,"[0.0, 0.0727411755655125, 0.07276464434895409,..."
3,0,100,1.000000e-12,0.078293,1.387779e-16,"[0.0, 0.08029310973608902, 0.08031128734148761..."
4,1,0,1.000000e-12,0.000005,6.938894e-17,"[0.0, 0.001999969005832517, 0.0039919067484825..."
5,1,1,1.000000e-12,0.000122,6.938894e-17,"[0.0, 0.0020086139642137685, 0.004008808854254..."
6,1,10,1.000000e-12,0.001185,8.326673e-17,"[0.0, 0.0020845968399092244, 0.004157299862984..."
7,1,100,1.000000e-12,0.008934,4.163336e-17,"[0.0, 0.002714862366541397, 0.0053842346310515..."
8,10,0,1.000000e-12,0.000005,1.110223e-16,"[0.0, 0.0019999659030612562, 0.003991900638662..."
9,10,1,1.000000e-12,0.000012,6.938894e-17,"[0.0, 0.0020008506949526716, 0.003993631009890..."
